In [1]:
import popy
from popy.inspector import NetworkInspector
from popy.pop_maker import PopMaker
import pandas as pd

In [2]:
model = popy.Model()

for _ in range(5):
    popy.Agent(model)

for _ in range(2):
    popy.Location(model)

model.locations[0].add_agents(model.agents[0:3])
model.locations[1].add_agents(model.agents[3:5])

assert len(model.agents) == 5
assert len(model.locations) == 2
assert len(model.locations[0].agents) == 3
assert len(model.locations[1].agents) == 2
assert len(model.agents[0].neighbors()) == 2
assert len(model.agents[-1].neighbors()) == 1

inspector = NetworkInspector(model)
inspector.plot_bipartite_network()

In [3]:
class ColorLocation(popy.Location):
    def __init__(self, model, color) -> None:
        super().__init__(model)
        self.color = color


class ColorAgent(popy.Agent):
    def __init__(self, model, color) -> None:
        super().__init__(model)
        self.color = color

    def change_location(self):
        for location in self.locations:
            if location.color != self.color:
                self.leave_location(location)
            
        for location in self.model.locations:
            if location.color == self.color and location not in self.locations:
                self.enter_location(location)
                return
        
        if self.color not in [location.color for location in self.model.locations]:
            location = ColorLocation(model=self.model, color=self.color)
            location.add_agent(self)
        
class ColorModel(popy.Model):
    def setup(self):
        for color in ["red", "green", "red", "blue", "blue", "red"]:
            ColorAgent(
                model=self, 
                color=color,
                )
        
        ColorLocation(self, "blue")
        ColorLocation(self, "red")

        self.locations[0].add_agents(self.agents[0:3])
        self.locations[1].add_agents(self.agents[3:6])

        assert len(self.agents) == 6
        assert len(self.locations) == 2
        assert len(self.locations[0].agents) == 3
        assert len(self.locations[1].agents) == 3

        self.inspector = NetworkInspector(self)
        self.inspector.plot_agent_network(node_attrs=["color"], node_color="color")

    def step(self):
        self.agents.change_location()
    
    def end(self):
        assert len(self.agents) == 6
        assert len(self.locations) == 3
        assert len(self.locations[0].agents) == 2
        assert len(self.locations[1].agents) == 3
        assert len(self.locations[2].agents) == 1
        self.inspector.plot_agent_network(node_attrs=["color"], node_color="color")

color_model = ColorModel()
color_model.run(steps=3)

Completed: 3 steps


Run time: 0:00:00.163033
Simulation finished


DataDict {
'info': Dictionary with 9 keys
'reporters': DataFrame with 1 variable and 1 row
}

In [4]:
class Town(popy.MagicLocation):
    size = 4

    def stick_together(self, agent):
        return agent.couple
    
    def assert_(self):
        assert len(self.agents) == 4
    
class Home(popy.MagicLocation):
    def split(self, agent):
        return agent.couple
    
    def weight(self, agent):
        return 12
    
    def assert_(self):
        assert len({a.couple for a in self.agents}) == 1
    
class Restaurant(popy.MagicLocation):
    def setup(self):
        chef = Chef(self.model)
        self.add_agent(chef)
        self.set_weight(agent=chef, weight=8)

    def weight(self, agent):
        return 2
    
    def split(self, agent):
        return agent.food

    def nest(self):
        return Town
    
    def assert_(self):
        # assert that affiliated agents are affiliated with the same Town
        assert len({
            agent.locations.select(agent.locations.cls == "Town")[0] 
            for agent in self.agents 
            if isinstance(agent, MyAgent) # Chef-agents are not affiliated with Towns
            }) == 1
    

class Chef(popy.Agent):
    def assert_(self):
        assert len(self.locations) == 1
        assert self.locations[0].cls == "Restaurant"
        assert self.get_location_weight(self.locations[0]) == 8
        assert len([1 for a in self.locations[0].neighbors(self) if self.get_agent_weight(a) != 2]) == 0

class MyAgent(popy.Agent):
    def assert_(self):
        couple_agent = [
            agent 
            for agent in self.model.agents.select(self.model.agents.cls == "MyAgent") 
            if agent.couple == self.couple and agent is not self
            ][0]
        
        assert (
            self.neighbors(location_classes=[Home])[0] is couple_agent
        )

        assert self.get_agent_weight(couple_agent) == 12 + 1
        
        assert (
            self.locations.select(self.locations.cls == "Town")[0] is self.shared_locations(couple_agent, location_classes=[Town])[0]
        )


df = pd.DataFrame(
    {
        "food": ["pizza", "pasta", "pizza", "pasta", "pizza", "pasta", "pizza", "pasta"],
        "couple": [1, 1, 2, 2, 3, 3, 4, 4],
        "age": [40, 40, 60, 60, 40, 40, 60, 60],
        }
)

model = popy.Model()
popmaker = PopMaker(model)
inspector = NetworkInspector(model)

popmaker.create_agents(df=df, agent_class=MyAgent)

popmaker.create_locations(
    location_classes=[
        Town,
        Home,
        Restaurant,
        ]
    )

inspector.plot_bipartite_network()
inspector.plot_agent_network()

model.agents.assert_()
model.locations.assert_()

[None, None, None, None, None, None, None, None, None, None]

In [18]:
import random
import math

from popy.agent import Agent

model = popy.Model()
popmaker = PopMaker(model)
inspector = NetworkInspector(model)

df = pd.DataFrame(
    {
        "food": ["pizza", "pasta", "pizza", "pasta", "pizza", "pasta", "pizza", "pasta"],
        "couple": [1, 1, 2, 2, 3, 3, 4, 4],
        "age": [40, 40, 60, 60, 40, 40, 60, 60],
        }
)


df = popmaker.draw_sample(df=df, n=20)

popmaker.create_agents(df=df)


    
class Table(popy.MagicLocation):
    recycle = True
    
    def melt(self):
        
        class PizzaGroup(popy.MagicLocation):
            size = 3
            exact_size_only = False

            def filter(self, agent):
                return agent.food == "pizza"
            
            #def split(self, agent):
            #    return agent.age
            
            def weight(self, agent):
                return 10

        class PastaGroup(popy.MagicLocation):
            size = 2
            exact_size_only = False
            
            def filter(self, agent):
                return agent.food == "pasta"
            
        return PizzaGroup, PastaGroup
    
    def nest(self):
        return Restaurant
    
    def weight(self, agent):
        return 5
    

class Restaurant(popy.MagicLocation):
    size = 10

    #def stick_together(self, agent):
    #    return agent.Table
    
    #def filter(self, agent):
    #    return agent.Table

popmaker.create_locations(
    location_classes=[
        Restaurant,
        Table,
        
        ]
    )

inspector.plot_agent_network(node_attrs=["food", "age", "couple"], node_color="food")
inspector.plot_bipartite_network()

In [24]:
def test_melt(
    size_pizza_group, 
    size_pasta_group, 
    recycle_, 
    exact_size_only_pizza_group, 
    exact_size_only_pasta_group,
    ):
    
    class Table(popy.MagicLocation):
        recycle = recycle_
        
        def melt(self):
            
            class PizzaGroup(popy.MagicLocation):
                size = size_pizza_group
                exact_size_only = exact_size_only_pizza_group

                def filter(self, agent):
                    return agent.food == "pizza"
                
                #def split(self, agent):
                #    return agent.age
                
                def weight(self, agent):
                    return 10

            class PastaGroup(popy.MagicLocation):
                size = size_pasta_group
                exact_size_only = exact_size_only_pasta_group
                
                def filter(self, agent):
                    return agent.food == "pasta"
                
            return PizzaGroup, PastaGroup

    df = pd.DataFrame({"food": ["pizza", "pasta"] * 10})
    model = popy.Model()

    popmaker = PopMaker(model)
    popmaker.create_agents(df=df)
    popmaker.create_locations(location_classes=[Table])

    return model

model = test_melt()

In [7]:
df = df.sample(frac=1).reset_index(drop=True)
df

,food,couple,age
0,pizza,2,60
1,pasta,3,40
2,pizza,4,60
3,pasta,2,60
4,pizza,3,40
5,pizza,1,40
6,pasta,3,40
7,pasta,1,40
8,pasta,4,60
9,pizza,1,40


In [8]:
class MyAgent(popy.Agent):
    def remove_friend(self):
        friends = self.neighbors(location_classes=[Friends])
        if friends:
            for friend in friends:
                if friend.opinion == self.opinion:
                    self.model.remove_locations(self.shared_locations(friend, location_classes=[Friends]))
                    break


    def get_friend(self):
        if len(self.neighbors(location_classes=[Friends])) == 0:
            for agent in self.model.agents:
                if agent.opinion != self.opinion and len(agent.neighbors(location_classes=[Friends])) == 0:
                    new_location = Friends(self.model)
                    new_location.add_agents([self, agent])
                    break
            

class Friends(popy.Location):
    size = 2

    def split(self, agent):
        return agent.opinion
    
    def weight(self, agent):
        return 10

class World(popy.Location):
    pass

    def weight(self, agent):
        return 0

class MyModel(popy.Model):
    def setup(self):
        self.inspector = NetworkInspector(self)
        self.popmaker = PopMaker(self)
        self.popmaker.create_agents(agent_class=MyAgent, df=df)
        self.popmaker.create_locations(location_classes=[
            World, 
            Friends,
            ])
        self.inspector.plot_agent_network(node_attrs=["opinion"], node_color="opinion")
    
    def step(self):
        self.agents.remove_friend()
        self.inspector.plot_agent_network(node_attrs=["opinion"], node_color="opinion")
        self.agents.get_friend()
        self.inspector.plot_agent_network(node_attrs=["opinion"], node_color="opinion")
        

model = MyModel()
model.run(steps=1)

AttributeError: World (Obj 1) has no attribute 'melt'.